In [9]:
import pandas as pd

tweets = pd.read_csv('data/tweets/all_tweets.csv')

tweets

,datetime,text,token
0,2022-08-29 02:18:35,first few days just 1k but going to 10k if no ...,"['first', 'few', 'days', 'just', '1k', 'but', ..."
1,2022-08-29 02:11:30,good to ask yourself this often,"['good', 'to', 'ask', 'yourself', 'this', 'oft..."
2,2022-08-29 02:11:12,agreed we recently made this change many an in...,"['agreed', 'we', 'recently', 'made', 'this', '..."
3,2022-08-28 23:51:49,this will happen again just a matter of time,"['this', 'will', 'happen', 'again', 'just', 'a..."
4,2022-08-28 22:54:36,the swoop of the x is meant to represent the a...,"['the', 'swoop', 'of', 'the', 'x', 'is', 'mean..."
...,...,...,...
13203,2018-01-07 03:00:48,near 405,"['near', '405']"
13204,2018-01-07 02:33:02,put an old school roller rock restaurant at on...,"['put', 'an', 'old', 'school', 'roller', 'rock..."
13205,2018-01-05 00:30:00,falcon heavy goes vertical,"['falcon', 'heavy', 'goes', 'vertical']"
13206,2018-01-03 08:22:31,a neural net to detect rain no rain or sun are...,"['a', 'neural', 'net', 'to', 'detect', 'rain',..."


In [10]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\levit\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [11]:
tweets['tokenized_text']= tweets['text'].apply(word_tokenize)

tweets['filtered_tokenized_text']= tweets['tokenized_text'].apply(lambda x: [word for word in x if word not in stopwords.words('english')])

tweets['filtered_text'] = tweets['filtered_tokenized_text'].apply((" ").join)

tweets


,datetime,text,token,tokenized_text,filtered_tokenized_text,filtered_text
0,2022-08-29 02:18:35,first few days just 1k but going to 10k if no ...,"['first', 'few', 'days', 'just', '1k', 'but', ...","[first, few, days, just, 1k, but, going, to, 1...","[first, days, 1k, going, 10k, major, note, 106...",first days 1k going 10k major note 10692 hopef...
1,2022-08-29 02:11:30,good to ask yourself this often,"['good', 'to', 'ask', 'yourself', 'this', 'oft...","[good, to, ask, yourself, this, often]","[good, ask, often]",good ask often
2,2022-08-29 02:11:12,agreed we recently made this change many an in...,"['agreed', 'we', 'recently', 'made', 'this', '...","[agreed, we, recently, made, this, change, man...","[agreed, recently, made, change, many, intense...",agreed recently made change many intense effor...
3,2022-08-28 23:51:49,this will happen again just a matter of time,"['this', 'will', 'happen', 'again', 'just', 'a...","[this, will, happen, again, just, a, matter, o...","[happen, matter, time]",happen matter time
4,2022-08-28 22:54:36,the swoop of the x is meant to represent the a...,"['the', 'swoop', 'of', 'the', 'x', 'is', 'mean...","[the, swoop, of, the, x, is, meant, to, repres...","[swoop, x, meant, represent, arc, orbit]",swoop x meant represent arc orbit
...,...,...,...,...,...,...
13203,2018-01-07 03:00:48,near 405,"['near', '405']","[near, 405]","[near, 405]",near 405
13204,2018-01-07 02:33:02,put an old school roller rock restaurant at on...,"['put', 'an', 'old', 'school', 'roller', 'rock...","[put, an, old, school, roller, rock, restauran...","[put, old, school, roller, rock, restaurant, o...",put old school roller rock restaurant one new ...
13205,2018-01-05 00:30:00,falcon heavy goes vertical,"['falcon', 'heavy', 'goes', 'vertical']","[falcon, heavy, goes, vertical]","[falcon, heavy, goes, vertical]",falcon heavy goes vertical
13206,2018-01-03 08:22:31,a neural net to detect rain no rain or sun are...,"['a', 'neural', 'net', 'to', 'detect', 'rain',...","[a, neural, net, to, detect, rain, no, rain, o...","[neural, net, detect, rain, rain, sun, patient]",neural net detect rain rain sun patient


In [37]:
tweets.to_csv('data/tweets/processed_all_tweets.csv', index=False)

In [38]:
tweets = pd.read_csv('data/tweets/processed_all_tweets.csv')
tweets

,datetime,text,token,tokenized_text,filtered_tokenized_text,filtered_text
0,2022-08-29 02:18:35,first few days just 1k but going to 10k if no ...,"['first', 'few', 'days', 'just', '1k', 'but', ...","['first', 'few', 'days', 'just', '1k', 'but', ...","['first', 'days', '1k', 'going', '10k', 'major...",first days 1k going 10k major note 10692 hopef...
1,2022-08-29 02:11:30,good to ask yourself this often,"['good', 'to', 'ask', 'yourself', 'this', 'oft...","['good', 'to', 'ask', 'yourself', 'this', 'oft...","['good', 'ask', 'often']",good ask often
2,2022-08-29 02:11:12,agreed we recently made this change many an in...,"['agreed', 'we', 'recently', 'made', 'this', '...","['agreed', 'we', 'recently', 'made', 'this', '...","['agreed', 'recently', 'made', 'change', 'many...",agreed recently made change many intense effor...
3,2022-08-28 23:51:49,this will happen again just a matter of time,"['this', 'will', 'happen', 'again', 'just', 'a...","['this', 'will', 'happen', 'again', 'just', 'a...","['happen', 'matter', 'time']",happen matter time
4,2022-08-28 22:54:36,the swoop of the x is meant to represent the a...,"['the', 'swoop', 'of', 'the', 'x', 'is', 'mean...","['the', 'swoop', 'of', 'the', 'x', 'is', 'mean...","['swoop', 'x', 'meant', 'represent', 'arc', 'o...",swoop x meant represent arc orbit
...,...,...,...,...,...,...
12958,2018-01-07 03:00:48,near 405,"['near', '405']","['near', '405']","['near', '405']",near 405
12959,2018-01-07 02:33:02,put an old school roller rock restaurant at on...,"['put', 'an', 'old', 'school', 'roller', 'rock...","['put', 'an', 'old', 'school', 'roller', 'rock...","['put', 'old', 'school', 'roller', 'rock', 're...",put old school roller rock restaurant one new ...
12960,2018-01-05 00:30:00,falcon heavy goes vertical,"['falcon', 'heavy', 'goes', 'vertical']","['falcon', 'heavy', 'goes', 'vertical']","['falcon', 'heavy', 'goes', 'vertical']",falcon heavy goes vertical
12961,2018-01-03 08:22:31,a neural net to detect rain no rain or sun are...,"['a', 'neural', 'net', 'to', 'detect', 'rain',...","['a', 'neural', 'net', 'to', 'detect', 'rain',...","['neural', 'net', 'detect', 'rain', 'rain', 's...",neural net detect rain rain sun patient


In [31]:
from wordcloud import WordCloud

long_string = ','.join(list(tweets['filtered_text'].values))

wordcloud = WordCloud(background_color="white", max_words=5000, contour_color='steelblue', width=800, height=800)

wordcloud.generate(long_string)

In [78]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

def get_top_n_words(corpus, n=None):
    vec = CountVectorizer(stop_words='english').fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:n]

    
common_words = get_top_n_words(tweets.filtered_text, 20)
unigram = pd.DataFrame(common_words, columns = ['unigram' , 'count'])

In [79]:
unigram

,unigram,count
0,yes,570
1,good,489
2,great,449
3,like,411
4,true,336
5,time,315
6,car,309
7,high,296
8,make,289
9,people,289


In [81]:
def get_top_n_trigram(corpus, n=None):
    vec = CountVectorizer(ngram_range=(3,3),stop_words='english').fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in      vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:n]
common_words = get_top_n_trigram(tweets.filtered_text, 10)
trigram = pd.DataFrame(common_words, columns = ['trigram' , 'count'])

In [82]:
trigram

,trigram,count
0,watch falcon launch,17
1,starship super heavy,13
2,great work team,12
3,super heavy booster,8
4,people dont know,8
5,team great work,7
6,falcon launch 53,6
7,launch 53 orbit,6
8,falcon 9s stage,6
9,easy production hard,6


In [83]:
from sklearn.decomposition import LatentDirichletAllocation
vectorizer = CountVectorizer(
analyzer='word',       
min_df=3,# minimum required occurences of a word 
stop_words='english',# remove stop words
lowercase=True,# convert all words to lowercase
token_pattern='[a-zA-Z0-9]{3,}',# num chars > 3
max_features=5000,# max number of unique words
                            )
data_matrix = vectorizer.fit_transform(tweets.filtered_text)
data_matrix

<12963x3174 sparse matrix of type '<class 'numpy.int64'>'
	with 55085 stored elements in Compressed Sparse Row format>

In [84]:
lda_model = LatentDirichletAllocation(
    n_components=10, # Number of topics
    learning_method='online',
    random_state=20,       
    n_jobs = -1 )


lda_output = lda_model.fit_transform(data_matrix)

In [87]:
import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()
pyLDAvis.sklearn.prepare(lda_model, data_matrix, vectorizer, mds='tsne')

c:\Users\levit\.pyenv\pyenv-win\versions\3.10.7\lib\site-packages\pyLDAvis\_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  by='saliency', ascending=False).head(R).drop('saliency', 1)


ValueError: perplexity must be less than n_samples

In [86]:
for i,topic in enumerate(lda_model.components_):
    print(f'Top 10 words for topic #{i}:')
    print([vectorizer.get_feature_names()[i] for i in topic.argsort()[-10:]])
    print('\n')

Top 10 words for topic #0:
['propellant', 'close', 'absolutely', 'fast', 'extremely', 'change', 'wow', 'amazing', 'work', 'sure']


Top 10 words for topic #1:
['china', 'build', 'end', 'super', 'day', 'working', 'lot', 'hard', 'thanks', 'thats']


Top 10 words for topic #2:
['test', 'heavy', 'orbit', 'future', 'rocket', 'engine', 'falcon', 'launch', 'probably', 'starship']


Top 10 words for topic #3:
['said', 'support', 'low', 'problem', 'times', 'long', 'people', 'right', 'production', 'good']


Top 10 words for topic #4:
['cool', 'love', 'point', 'best', 'team', 'new', 'way', 'maybe', 'model', 'great']


Top 10 words for topic #5:
['buy', 'bad', 'getting', 'ago', 'year', 'real', 'know', 'big', 'coming', 'yes']


Top 10 words for topic #6:
['energy', 'people', 'power', 'going', 'use', 'think', 'better', 'high', 'dont', 'make']


Top 10 words for topic #7:
['early', 'version', 'safety', 'later', 'release', 'beta', 'mass', 'week', 'need', 'true']


Top 10 words for topic #8:
['air', 'd

c:\Users\levit\.pyenv\pyenv-win\versions\3.10.7\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
